In [1]:
import theano
import numpy as np
import theano.tensor as T

In [87]:
x = np.array([[0,1,3,5,6,-1], [0,5,6,7,8,-1], [4,2,1,4,5,-1]])
most_common = [1,2,5]
vocab_size = x.max()

In [17]:
from collections import Counter
c = Counter([1,2,4,4])

[1, 2, 4]

In [101]:
def frequency_vec(x, most_common, vocab_size):
    
    def array_to_list(array, max_index):
        c = Counter(array[array > 0])
        l = np.zeros((max_index))
        l[list(c.keys())] = list(c.values())
        return l
    
    khot = np.array(list(map(lambda row: array_to_list(row, vocab_size+1), x)))
    if most_common is None:
        return khot
    else:
        return np.delete(arr=khot, axis=1, obj=most_common)
    
def theano_frequency_vec():
    
    X = T.matrix('X')
    ret = frequency_vec(X, most_common, vocab_size)
    
    return theano.function(inputs=[x], outputs=ret, allow_input_downcast=True)

In [102]:
f = theano_frequency_vec()

TypeError: TensorType does not support iteration. Maybe you are using builtin.sum instead of theano.tensor.sum? (Maybe .max?)

In [93]:
frequency_vec(x, most_common, vocab_size)

array([[ 0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  2.,  0.,  0.,  0.]])

In [22]:
def func(x, a=3, b=3):
    return (x+a)**b

In [23]:
x = T.scalar('x')
out = func(x)
f = theano.function([x], out)

In [93]:
def get_func(*args):
    a = args[0]
    b = args[1]
    
    x = T.scalar('x')
    out = func(x, a, b)
    return theano.function(inputs=[x], outputs=out)

In [94]:
f = get_func(3,4)

In [95]:
f(4)

array(2401.0, dtype=float32)

In [98]:
def get_another_func(*args):
    f1 = args[0]
    
    z = T.scalar('z')
    out = np.sqrt(z + f1)
    return theano.function(inputs=[z], outputs=out, on_unused_input='ignore')

In [101]:
f2 = get_another_func(f(4))

In [102]:
f2(2)

array(49.02040481567383, dtype=float32)

In [35]:
np.sqrt(2+(4+3)**4)

49.020403915104573

In [36]:
def get_another_func_var(*args):
    f = args[0]
    x = args[1]
    
    f1 = f(x)
    z = T.scalar('z')
    out = np.sqrt(z + f1)
    return theano.function(inputs=[z], outputs=out)

In [39]:
f3 = get_another_func_var(f, 4)

In [42]:
f3(2)

array(49.02040481567383, dtype=float32)

In [64]:
def g1():    
    x = T.vector('x')
    y = T.scalar('y')
    z = x * T.shape_padright(y)
    return theano.function([x,y], z, allow_input_downcast=True)

In [86]:
f = g1()

In [67]:
g(np.array([3,4,5]), 2)

array([  6.,   8.,  10.], dtype=float32)

In [87]:
def g2():
    z = T.scalar('z')
    w = T.scalar('w')
    j = f(z) ** w
    return theano.function([z,w], j, allow_input_downcast=True)

In [22]:
import theano
import theano.tensor as T

import numpy as np

from lasagne.layers import InputLayer, LSTMLayer, RecurrentLayer, BiasLayer, NonlinearityLayer, ConcatLayer, \
ElemwiseMergeLayer, get_output
from lasagne.nonlinearities import sigmoid, softmax, tanh

l_in = InputLayer((None, 40, 300))

l_current = l_in

for h in range(4):

    l_current = LSTMLayer(l_current, num_units=500, nonlinearity=tanh)

l_out = RecurrentLayer(l_current, num_units=300, W_hid_to_hid=T.zeros, nonlinearity=None)

l_in_gate = InputLayer((None, 40, 300))

l_bias = BiasLayer(l_out)
l_sigm = NonlinearityLayer(l_bias, nonlinearity=sigmoid)

def gate():
    x = T.dtensor3('x')
    y = T.dtensor3('y')
    gate = (1 - x) * y
    
    return theano.function([x, y], gate, allow_input_downcast=True)

gate_fn = gate()

l_gate = ElemwiseMergeLayer([l_sigm, l_in_gate], merge_function=gate_fn)

l_concat = ConcatLayer([l_out, l_gate], axis=0)
l_final = BiasLayer(l_concat)

In [23]:
data = np.random.randn(40, 300)
fake_data = np.random.randn(30, 300)

In [24]:
get_output(l_final, data)

ValueError: get_output() was called with a single input expression on a network with multiple input layers. Please call it with a dictionary of input expressions instead.

In [7]:
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import theano
import theano.tensor as T
import numpy as np
x = T.imatrix()
l_in = InputLayer((3, ))
W = np.arange(3*5).reshape((3, 5)).astype('float32')
l1 = EmbeddingLayer(l_in, input_size=3, output_size=5, W=W)
output = get_output(l1, x)
f = theano.function([x], output)
x_test = np.array([[0, 2], [1, 2]]).astype('int32')
print(f(x_test).shape)
f(x_test)

(2, 2, 5)


array([[[  0.,   1.,   2.,   3.,   4.],
        [ 10.,  11.,  12.,  13.,  14.]],

       [[  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.]]], dtype=float32)